<a href="https://colab.research.google.com/github/ClintJensen/CovNet_Human_Drawings/blob/master/NEXT_images_dap_feature_extraction_all_305.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature extraction from all kids images with block5 max pool layer

In [1]:
# Here we use tensorflow.keras API to process the data by using VGG19 
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model

# to import filenames
from os import listdir
from os.path import isfile, join

# to handle data
import numpy as np

In [2]:
# Load base model with imagenet pre-trained weights 
base_model = VGG19(weights='imagenet')

574717952/574710816 [==============================] - 11s 0us/step


In [3]:
# Here we can see the progression from layer to layer 
# The output shape column shows how the image gets compressed as 
# it pass through the layers
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Feature extraction for a single image with block5 max pool layer

In [4]:
####################################
# To get the data If you're in Colab
####################################

# Images are storage in GitHub. By running this we clone the data into Colab
! git clone https://github.com/ClintJensen/CovNet_Human_Drawings
# Run this just once per sesion

# Now repo data is available in Colab local environment
!ls CovNet_Human_Drawings/data

Cloning into 'CovNet_Human_Drawings'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 1332 (delta 86), reused 0 (delta 0), pack-reused 1200
Receiving objects: 100% (1332/1332), 119.94 MiB | 32.21 MiB/s, done.
Resolving deltas: 100% (499/499), done.
Checking out files: 100% (1490/1490), done.
cluster-dataframes   dap-metadata-kids	merged-dataframes-prediction
dap-drawings-adults  DAP_NEXT_Images	optimization-results
dap-drawings-all     edit-csv		README.md
dap-drawings-kids    edit-filenames	vectors-features


In [5]:
ls

CovNet_Human_Drawings/  sample_data/


In [6]:
#/content/CovNet_Human_Drawings/data/DAP_NEXT_Images
# Print filenames on DAP_NEXT_Images
!ls ../content/CovNet_Human_Drawings/data/DAP_NEXT_Images | head

DAM001_F_DAPedit.jpg
DAM001_P_DAPedit.jpg
DAM001_T_DAPedit.jpg
DAM002_F_DAPedit.jpg
DAM002_P_DAPedit.jpg
DAM002_T_DAPedit.jpg
DAM003_F_DAPedit.jpg
DAM003_P_DAPedit.jpg
DAM003_T_DAPedit.jpg
DAM004_F_DAPedit.jpg


In [7]:
# Load and reshape the image to be feed into the model
img_path = '../content/CovNet_Human_Drawings/data/DAP_NEXT_Images/DAM001_F_DAPedit.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [8]:
#######################################
# Select a layer for feature extraction
#######################################

# Here we pick the maxpooling layer in block 5
feature_layer = "block5_pool"

# To check other layers
# feature_layer = "block1_pool "
# feature_layer = "block1_conv1 "
 
model = Model(inputs=base_model.input, 
              outputs=base_model.get_layer(feature_layer).output)

In [9]:
#####################################################
# Do the feature extraction with block5 pooling layer
#####################################################

block5_pool_features = model.predict(x)

In [10]:
# In block 5, we can check that feature has the same shape that the maxpooling 
# layer in block 5 (above drawing)

# Print tensor shape
block5_pool_features.shape

(1, 7, 7, 512)

In [11]:
# Print extracted feature as a tensor (i.e., feature)
#print(block5_pool_features)

[[[[ 0.         0.         0.        ...  0.         3.044863
     0.       ]
   [ 0.         0.         0.        ...  0.         0.7641097
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         1.377389
     0.       ]]

  [[ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 1.8787315  0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]]

  [[ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         

In [12]:
# Print extracted feature flattened as a 1D vector
vgg19_feature_np = np.array(block5_pool_features)
vgg19_feature_np.flatten()

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

## Feature extraction from ALL images with block5 max pool layer

In [13]:
# Get the drawings filenames from directory 

# If relative path doesn't work, change path as nedeed
path = '../content/CovNet_Human_Drawings/data/DAP_NEXT_Images/'
filenames = [f for f in listdir(path) if isfile(join(path, f))]
len(filenames) # This should yield 305

305

In [14]:
#########################################
# Select the layer for feature extraction
#########################################

# A list of the layers' names is obtained by running "base_model.summary()"
feature_layer = "block5_pool" # let's take the last max pool as example
model = Model(inputs=base_model.input, 
              outputs=base_model.get_layer(feature_layer).output)

In [15]:
##########################################
# Do the feature extraction for all images
##########################################

# Let's create a list to save flattened tensors as vectors
vgg19_feature_list = []

# Loop over filenames and append flattened tensor to vector list
for fname in filenames:
  # This part of the loop reshapes and preprocesses the input images 
  img_path = path + fname
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  
  # This part of the loop extracts the features and flattens the tensors to vectors
  vgg19_feature = model.predict(x)
  vgg19_feature_np = np.array(vgg19_feature)
  vgg19_feature_list.append(vgg19_feature_np.flatten())

In [16]:
##############################################
# Pass the VGG19 feature list to a numpy array
##############################################

vgg19_feature_list_np = np.array(vgg19_feature_list)
print(type(vgg19_feature_list_np))
print(vgg19_feature_list_np.shape)

<class 'numpy.ndarray'>
(305, 25088)


In [17]:
###################################
# Export numpy array as a .npy file
###################################

#.npy files are lightweight and easier to load back on python

save_path = '../content/CovNet_Human_Drawings/data/vectors-features/'
filename = 'vgg19_vectors_drawings_block5_pool_all_305_raw'
np.save(save_path+filename, vgg19_feature_list_np)

In [18]:
##################################
# Export numpy array as a csv file
##################################

# If you save np arrays as .csv files, they will be very heavy
# zip files afterwards to avoid conflicts on GitHub (100mg limit upstream)

save_path = '../content/CovNet_Human_Drawings/data/vectors-features/'
filename = 'vgg19_vectors_drawings_block5_pool_all_305_raw.csv'
np.savetxt(save_path+filename, vgg19_feature_list_np, delimiter=",")